In [1]:
# Importing the required library
import pandas as pd

# Reading the Excel file into a Pandas DataFrame
df = pd.read_excel('Data/Data_BA.xlsx')

# Displaying the first 5 rows of the DataFrame to verify that the data has been read correctly
df


,Område,Team,Indsatslængde,Indsats,Indsatsparagraf,Leverandør,PrisIAlt,Underretter,Haendelsesdato,BaggrundBarnet,BaggrundHjemmet,alder,alder_ved_hændelse,sag_start,sag_slut,status,personID_serial,caseworker_serial,sag_varighed
0,BBU City-Østerbro,BBU City-Østerbro - MY Børnegruppen,90,Familiebehandling,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Foranstaltningshus BBU City-Østerbro,28091,Daginstitution/Dagpleje,2020-07-14,Anden bekymrende adfærd hos barn/ung fx udadre...,Højt konfliktniveau eller vold i hjemmet melle...,6,3,2020-07-14,2021-12-02,lukket,629,1,507
1,BBU City-Østerbro,BBU City-Østerbro - MY Børnegruppen,161,Familiebehandling,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Foranstaltningshus BBU City-Østerbro,47375,Anden kommunal afdeling,2020-09-18,NaN,NaN,2,0,2020-09-18,2022-12-07,lukket,604,1,811
2,BBU City-Østerbro,BBU City-Østerbro - MY Børnegruppen,0,Familiebehandling,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Børne- og Familiehuset Arild,39,Sygehus,2021-05-21,Anden form for omsorgssvigt over for barn/ung¤...,Anden bekymrende adfærd hos forældre¤Misbrug h...,1,0,2021-05-21,2023-09-15,lukket,34,1,848
3,BBU Valby-Vesterbro-Kgs.Enghave,BBU Valby-Vesterbro-Kgs.Enghave - Styrket modt...,239,Familiebehandling,"SEL § 52, stk. 3, nr. 3 (fx familiebehandling)",Foranstaltningshus BBU Valby-Vesterbro-Kgs.Eng...,141911,Anden offentlig myndighed,2020-03-05,Anden form for omsorgssvigt over for barn/ung,Misbrug hos forældre¤Utilstrækkelig omsorg fra...,2,0,2020-03-05,2021-10-18,lukket,167,2,593
4,BBU AF Økonomi og Digitalisering,NaN,123,Psykologisk behandling,"SEL § 52, stk. 3, nr. 3 (behandling af barn/un...",Psykolog Tanja Skov Rønne,49000,Politi,2021-01-23,NaN,Betydelig eller varigt nedsat fysisk eller psy...,5,3,2021-01-23,2023-02-17,lukket,671,3,756


In [ ]:
# Removing rows where 'PrisIAlt' is NaN (Not a Number) and 'status' is 'åben'
df.dropna(subset=['PrisIAlt'], inplace=True)
df = df[df['status'] != 'åben']

# Displaying the first few rows of the modified DataFrame to verify the changes
df

In [ ]:
# Save the modified DataFrame to a new Excel file
df.to_excel('Data/Data_BA_modified.xlsx', index=False)

In [2]:
# Group by 'Område' and compute the average price for each
average_price_by_område = data.groupby('Område')['PrisIAlt'].mean().sort_values(ascending=False)

average_price_by_område


Område
Teamet for koordineret indsats (KIT)    149240.000000
BBU Valby-Vesterbro-Kgs.Enghave          98769.597015
BCH Børneområdet                         97714.956522
BBU Brønshøj-Husum-Vanløse               94237.909091
BBU City-Østerbro                        93842.218182
BBU Amager                               71151.040816
BBU Nørrebro-Bispebjerg                  65864.847716
BBU AF Økonomi og Digitalisering         49000.000000
BBU Flygtningeindsats                    27722.000000
Name: PrisIAlt, dtype: float64

In [3]:
# One-hot encode the categorical control variables
encoded_data = pd.get_dummies(data, columns=['Indsats', 'Underretter', 'BaggrundBarnet', 'BaggrundHjemmet'], drop_first=True)

# Check the shape of the cleaned and encoded data
encoded_data.shape


(651, 95)

In [5]:
# 1. Identify the columns that contain the ¤ symbol
cols_with_symbol = [col for col in encoded_data.columns if '¤' in col]

# 2. For each identified column, split its values based on the ¤ symbol and create new columns
for col in cols_with_symbol:
    # Split the column name by the ¤ symbol
    split_names = col.split('¤')
    
    # For each split name, create a new column if it doesn't exist
    for name in split_names:
        new_col_name = col.split('_')[0] + '_' + name
        if new_col_name not in encoded_data.columns:
            encoded_data[new_col_name] = 0

    # Transfer the values from the original column to the new columns without overwriting
    for name in split_names:
        new_col_name = col.split('_')[0] + '_' + name
        mask = (encoded_data[col] == 1)
        encoded_data.loc[mask, new_col_name] = encoded_data.loc[mask, new_col_name] | encoded_data.loc[mask, col]

# 3. Drop the original columns that contained the ¤ symbol
encoded_data.drop(columns=cols_with_symbol, inplace=True)

# Check the resulting dataframe
print(encoded_data.columns)


Index(['Område', 'Team', 'Indsatslængde', 'Indsatsparagraf', 'Leverandør',
       'PrisIAlt', 'Haendelsesdato', 'alder', 'alder_ved_hændelse',
       'sag_start', 'sag_slut', 'status', 'personID_serial',
       'caseworker_serial', 'sag_varighed', 'Indsats_Kontaktperson',
       'Indsats_Pasning', 'Indsats_Psykologisk behandling',
       'Indsats_Socialpædagogisk behandling', 'Underretter_Anbringelsessted',
       'Underretter_Anden kommunal afdeling',
       'Underretter_Anden offentlig myndighed', 'Underretter_Anonym',
       'Underretter_Barnets netværk', 'Underretter_Daginstitution',
       'Underretter_Daginstitution/Dagpleje', 'Underretter_Døgnvagt',
       'Underretter_Familie', 'Underretter_Familiebehandling',
       'Underretter_Familiens bekendtskabskreds',
       'Underretter_Forældremyndighedsindehaver', 'Underretter_Kontaktperson',
       'Underretter_Krisecenter', 'Underretter_Læge',
       'Underretter_Modtaget fra anden kommune', 'Underretter_Naboer',
       'Underrette

In [ ]:
# Define the control variables
control_columns = ['Indsats', 'Underretter', 'BaggrundBarnet', 'Baggrundhjemmet', 'alder_ved_hændelse', 'caseworker_serial']
controls_df = encoded_data[control_columns]

# Add the predicted price and control variables to the second stage regression
X_second_stage = sm.add_constant(pd.concat([encoded_data['predicted_price'], controls_df], axis=1))
y = encoded_data['sag_varighed']

second_stage = sm.OLS(y, X_second_stage).fit()

second_stage.summary()

